In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 32
compression = 0.5
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 17s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 32, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 32, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 32
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 312s 6ms/step - loss: 1.4764 - acc: 0.4555 - val_loss: 2.4509 - val_acc: 0.3712
Epoch 2/50
19200/50000 [==========>...................] - ETA: 2:50 - loss: 1.1288 - acc: 0.593450000/50000 [==============================] - 296s 6ms/step - loss: 1.0599 - acc: 0.6193 - val_loss: 1.2210 - val_acc: 0.6091
Epoch 3/50
45568/50000 [==========================>...] - ETA: 24s - loss: 0.8994 - acc: 0.679050000/50000 [==============================] - 296s 6ms/step - loss: 0.8956 - acc: 0.6802 - val_loss: 1.4691 - val_acc: 0.5639
Epoch 4/50
50000/50000 [==============================] - 296s 6ms/step - loss: 0.7883 - acc: 0.7196 - val_loss: 1.8233 - val_acc: 0.5262
Epoch 5/50
 3328/50000 [>.............................] - ETA: 4:18 - loss: 0.7395 - acc: 0.737150000/50000 [==============================] - 296s 6ms/step - loss: 0.7099 - acc: 0.7505 - val_loss: 1.0765 - val_acc: 0.6862
Epoch 

 1536/50000 [..............................] - ETA: 4:27 - loss: 0.1272 - acc: 0.957750000/50000 [==============================] - 295s 6ms/step - loss: 0.1484 - acc: 0.9469 - val_loss: 0.6816 - val_acc: 0.8388
Epoch 45/50
38528/50000 [======================>.......] - ETA: 1:03 - loss: 0.1431 - acc: 0.949050000/50000 [==============================] - 296s 6ms/step - loss: 0.1484 - acc: 0.9468 - val_loss: 0.9658 - val_acc: 0.7964
Epoch 46/50
50000/50000 [==============================] - 295s 6ms/step - loss: 0.1477 - acc: 0.9470 - val_loss: 0.6078 - val_acc: 0.8573
Epoch 47/50
 1536/50000 [..............................] - ETA: 4:28 - loss: 0.1481 - acc: 0.948650000/50000 [==============================] - 295s 6ms/step - loss: 0.1444 - acc: 0.9490 - val_loss: 0.7548 - val_acc: 0.8380
Epoch 48/50
38528/50000 [======================>.......] - ETA: 1:03 - loss: 0.1342 - acc: 0.952750000/50000 [==============================] - 296s 6ms/step - loss: 0.1376 - acc: 0.9508 - val_loss: 0.

In [13]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 26s 3ms/step
Test loss: 0.7562828681230545
Test accuracy: 0.8371


In [14]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk
